In [3]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey=demo"
r = requests.get(url)
data = r.json()

print(data)

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events', '2. Symbol': 'IBM', '3. Last Refreshed': '2025-12-18', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-12-18': {'1. open': '303.01', '2. high': '304.33', '3. low': '295.7', '4. close': '300.37', '5. adjusted close': '300.37', '6. volume': '4021048', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2025-12-17': {'1. open': '303.79', '2. high': '306.25', '3. low': '303.08', '4. close': '303.32', '5. adjusted close': '303.32', '6. volume': '3130954', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2025-12-16': {'1. open': '307.315', '2. high': '307.38', '3. low': '300.42', '4. close': '303.18', '5. adjusted close': '303.18', '6. volume': '3366878', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2025-12-15': {'1. open': '308.985', '2. high': '311.3629', '3. low': '306.35', '4. close': '308.66', '5. adjusted close'

In [ ]:
data["Time Series (Daily)"]["2025-12-18"]

{'1. open': '303.01',
 '2. high': '304.33',
 '3. low': '295.7',
 '4. close': '300.37',
 '5. adjusted close': '300.37',
 '6. volume': '4021048',
 '7. dividend amount': '0.0000',
 '8. split coefficient': '1.0'}

: 